In [31]:
import pandas as pd


# Read the parquet file into a DataFrame
df = pd.read_parquet('/Users/harshitgupta/Desktop/vs /VS-Data-Den/model_training/multivariate/sorted_data.parquet')



In [32]:
df.columns

Index(['PTNR_RM_CODE', 'WEEK', 'DATE_FIELD', 'SALES', 'FLAG', 'yearweeks'], dtype='object')

In [33]:
df.drop(['yearweeks'], axis=1, inplace=True)

In [34]:
df.columns

Index(['PTNR_RM_CODE', 'WEEK', 'DATE_FIELD', 'SALES', 'FLAG'], dtype='object')

In [35]:
df['DATE_FIELD'] = pd.to_datetime(df['DATE_FIELD'])  # Convert to datetime if not already in datetime format
df['YEARWEEKS'] = df['DATE_FIELD'].dt.strftime('%Y-%U')

# If you want to convert the YEARWEEKS to integer format
df['YEARWEEKS'] = df['DATE_FIELD'].dt.strftime('%Y%U').astype(int)

In [36]:
df.drop(['DATE_FIELD'], axis=1, inplace=True)

In [37]:
df.columns

Index(['PTNR_RM_CODE', 'WEEK', 'SALES', 'FLAG', 'YEARWEEKS'], dtype='object')

In [43]:
def fill_missing_yearweeks(group):
    min_yearweek = group['YEARWEEKS'].astype(int).min()
    max_yearweek = group['YEARWEEKS'].astype(int).max()
    all_yearweeks = [str(yw) for yw in range(min_yearweek, max_yearweek + 1)]
    missing_yearweeks = set(all_yearweeks) - set(group['YEARWEEKS'])
    missing_data = []
    for yw in missing_yearweeks:
        week = int(yw[-2:])  # Extract week from yearweek
        missing_data.append({'PTNR_RM_CODE': group['PTNR_RM_CODE'].iloc[0],
                             'WEEK': week,
                             'SALES': 0,
                             'YEARWEEKS': yw})
    return pd.DataFrame(missing_data)


In [44]:
filled_data = df.groupby('PTNR_RM_CODE').apply(fill_missing_yearweeks)

/var/folders/_p/ljb0l43112vc95swz8nhgv340000gn/T/ipykernel_52267/3338041370.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_data = df.groupby('PTNR_RM_CODE').apply(fill_missing_yearweeks)


In [ ]:
df_filled = pd.concat([df, filled_data], ignore_index=True)

In [41]:
df_filled.sort_values(by=['PTNR_RM_CODE', 'YEARWEEKS'], inplace=True)

In [42]:
grouped_data = df_filled.groupby(['PTNR_RM_CODE', 'YEARWEEKS']).sum().reset_index()